In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math

from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

%matplotlib inline

In [2]:
AdvWorksTest = pd.read_csv('AW_test.csv')
AdvWorksTrain = pd.read_csv('AdvWorksCusts.csv')
AW_BikeBuyer = pd.read_csv('AW_BikeBuyer.csv')

AdvWorksTest.columns = [str.replace('-', '_') for str in AdvWorksTest.columns]
AdvWorksTrain.columns = [str.replace('-', '_') for str in AdvWorksTrain.columns]

for col in AdvWorksTest.columns:
    if AdvWorksTest[col].dtype == object:
        count = 0
        count = [count + 1 for x in AdvWorksTest[col] if x == '?']
        #print(col + ' ' + str(sum(count)))
        
for col in AdvWorksTrain.columns:
    if AdvWorksTrain[col].dtype == object:
        count = 0
        count = [count + 1 for x in AdvWorksTrain[col] if x == '?']
        #print(col + ' ' + str(sum(count)))    
        
AdvWorksTrain = pd.merge(AdvWorksTrain, AW_BikeBuyer)        
AdvWorksTest = pd.merge(AdvWorksTest, AW_BikeBuyer)        


print(AdvWorksTrain.shape)
print(AdvWorksTest.shape)  

(16749, 24)
(4, 24)


In [3]:
AdvWorksTest.drop('Title', axis = 1, inplace = True)
AdvWorksTest.drop('Suffix', axis = 1, inplace = True)
AdvWorksTest.drop('AddressLine2', axis = 1, inplace = True)
AdvWorksTest.drop('MiddleName', axis = 1, inplace = True)
AdvWorksTrain.drop('Title', axis = 1, inplace = True)
AdvWorksTrain.drop('Suffix', axis = 1, inplace = True)
AdvWorksTrain.drop('AddressLine2', axis = 1, inplace = True)
AdvWorksTrain.drop('MiddleName', axis = 1, inplace = True)

print(AdvWorksTrain.shape)
print(AdvWorksTest.shape)

(16749, 20)
(4, 20)


In [4]:
AdvWorksTrain.head(5)

,CustomerID,FirstName,LastName,AddressLine1,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,BikeBuyer
0,11000,Jon,Yang,3761 N. 14th St,Rockhampton,Queensland,Australia,4700,1 (11) 500 555-0162,1966-04-08,Bachelors,Professional,M,M,1,0,0,2,137947,0
1,11001,Eugene,Huang,2243 W St.,Seaford,Victoria,Australia,3198,1 (11) 500 555-0110,1965-05-14,Bachelors,Professional,M,S,0,1,3,3,101141,1
2,11002,Ruben,Torres,5844 Linden Land,Hobart,Tasmania,Australia,7001,1 (11) 500 555-0184,1965-08-12,Bachelors,Professional,M,M,1,1,3,3,91945,0
3,11003,Christy,Zhu,1825 Village Pl.,North Ryde,New South Wales,Australia,2113,1 (11) 500 555-0162,1968-02-15,Bachelors,Professional,F,S,0,1,0,0,86688,0
4,11004,Elizabeth,Johnson,7553 Harness Circle,Wollongong,New South Wales,Australia,2500,1 (11) 500 555-0131,1968-08-08,Bachelors,Professional,F,S,1,4,5,5,92771,1


In [5]:
AdvWorksTrain_counts = AdvWorksTrain[['CustomerID','BikeBuyer']].groupby('BikeBuyer').count()
print(AdvWorksTrain_counts)

           CustomerID
BikeBuyer            
0               11188
1                5561


In [6]:
labels = np.array(AdvWorksTrain['BikeBuyer'])
print(labels)

[0 1 0 ... 1 0 0]


In [7]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Gender','MaritalStatus']

Features = encode_string(AdvWorksTrain['Occupation'])
for col in categorical_columns:
    temp = encode_string(AdvWorksTrain[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)
print(Features[:2, :])   

(16749, 9)
[[0. 0. 0. 1. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 1.]]


In [8]:
Features = np.concatenate([Features, np.array(AdvWorksTrain[['NumberCarsOwned','NumberChildrenAtHome'
                                                              ,'YearlyIncome','TotalChildren']])], axis = 1)
print(Features.shape)
print(Features[:2, :])   

(16749, 13)
[[0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00
  1.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.37947e+05
  2.00000e+00]
 [0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00
  1.00000e+00 0.00000e+00 1.00000e+00 1.00000e+00 3.00000e+00 1.01141e+05
  3.00000e+00]]


In [9]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 300)

X_train = Features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
X_test = Features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [10]:
scaler = preprocessing.StandardScaler().fit(X_train[:,12:])

In [11]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
probabilities = logistic_mod.predict_proba(X_test)

In [13]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.80) 

In [14]:
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])

  
print_metrics(y_test, scores)     

                 Confusion matrix
                 Score positive    Score negative
Actual positive       214                 0
Actual negative        86                 0

Accuracy  0.71
 
           Positive      Negative
Num case      214            86
Precision    0.71          0.00
Recall       1.00          0.00
F1           0.83          0.00


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
AdvWorksTest = pd.read_csv('AW_test.csv')

In [16]:
AdvWorksTest.drop('Title', axis = 1, inplace = True)
AdvWorksTest.drop('Suffix', axis = 1, inplace = True)
AdvWorksTest.drop('AddressLine2', axis = 1, inplace = True)
AdvWorksTest.drop('MiddleName', axis = 1, inplace = True)

#AdvWorksTest = pd.merge(AdvWorksTest, AW_BikeBuyer)  
print(AdvWorksTest.shape)

(500, 19)


In [17]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Gender','MaritalStatus']

FeaturesTest = encode_string(AdvWorksTest['Occupation'])
for col in categorical_columns:
    temp = encode_string(AdvWorksTest[col])
    FeaturesTest = np.concatenate([FeaturesTest, temp], axis = 1)

print(FeaturesTest.shape)
print(FeaturesTest[:2, :])   

(500, 9)
[[0. 1. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 1. 1. 0.]]


In [18]:
FeaturesTest = np.concatenate([FeaturesTest, np.array(AdvWorksTest[['NumberCarsOwned','NumberChildrenAtHome'
                                                              ,'YearlyIncome','TotalChildren']])], axis = 1)
print(FeaturesTest.shape)
print(FeaturesTest[:2, :])   

(500, 13)
[[0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00
  0.00000e+00 0.00000e+00 1.00000e+00 2.00000e+00 0.00000e+00 8.69310e+04
  5.00000e+00]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00
  1.00000e+00 1.00000e+00 0.00000e+00 2.00000e+00 2.00000e+00 1.00125e+05
  4.00000e+00]]


In [19]:
logistic_mod.predict(FeaturesTest)

array([0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,